In [8]:
import struct
from typing import Any, Dict, List, Tuple

def encode(data: Any) -> bytes:
    if data is None:
        return b"\x00"  # Null value
    elif isinstance(data, int):
        # Encode integer as 8 bytes
        encoded = b"\x01" + data.to_bytes(8, byteorder="little")
        print(f"Encoding integer: {data} -> {encoded}")
        return encoded
    elif isinstance(data, bytes):
        # Encode bytes with length prefix
        encoded = b"\x02" + len(data).to_bytes(1, byteorder="little") + data
        print(f"Encoding bytes: {data} -> {encoded}")
        return encoded
    elif isinstance(data, str):
        # Encode string with UTF-8 and length prefix
        encoded_str = data.encode("utf-8")
        encoded = b"\x03" + len(encoded_str).to_bytes(1, byteorder="little") + encoded_str
        print(f"Encoding string: {data} -> {encoded}")
        return encoded
    elif isinstance(data, list):
        # Encode list recursively
        encoded_items = b"".join(encode(item) for item in data)
        encoded = b"\x04" + len(data).to_bytes(1, byteorder="little") + encoded_items
        print(f"Encoding list: {data} -> {encoded}")
        return encoded
    elif isinstance(data, dict):
        # Encode dict in lexicographical key order
        encoded_pairs = b""
        for key in sorted(data.keys()):
            encoded_key = encode(key)
            encoded_value = encode(data[key])
            encoded_pairs += encoded_key + encoded_value
        encoded = b"\x05" + len(data).to_bytes(1, byteorder="little") + encoded_pairs
        print(f"Encoding dictionary: {data} -> {encoded}")
        return encoded
    else:
        raise TypeError(f"Unsupported data type: {type(data)}")

def decode(data: bytes) -> Any:

    def decode_next(data: bytes, index: int) -> Tuple[Any, int]:
        marker = data[index]
        index += 1

        if marker == 0x00:  # None
            return None, index
        elif marker == 0x01:  # Integer
            value = int.from_bytes(data[index:index + 8], byteorder="little")
            return value, index + 8
        elif marker == 0x02:  # Bytes
            length = data[index]
            index += 1
            value = data[index:index + length]
            return value, index + length
        elif marker == 0x03:  # String
            length = data[index]
            index += 1
            value = data[index:index + length].decode("utf-8")
            return value, index + length
        elif marker == 0x04:  # List
            length = data[index]
            index += 1
            items = []
            for _ in range(length):
                item, index = decode_next(data, index)
                items.append(item)
            return items, index
        elif marker == 0x05:  # Dictionary
            length = data[index]
            index += 1
            dictionary = {}
            for _ in range(length):
                key, index = decode_next(data, index)
                value, index = decode_next(data, index)
                dictionary[key] = value
            return dictionary, index
        else:
            raise ValueError(f"Unknown marker: {marker}")

    result, _ = decode_next(data, 0)
    return result

# Test Input
INPUT_1 = {
    "null": None,
    "octets": bytes([1, 2, 3]),
    "integer": 12345
}

# Updated Expected Output
EXPECTED_1 = b'\x05\x03\x03\x07integer\x0190\x00\x00\x00\x00\x00\x00\x03\x04null\x00\x03\x06octets\x02\x03\x01\x02\x03'

# Encoding Test
encoded_data_1 = encode(INPUT_1)
print(f"\nEncoded Data: {encoded_data_1}")
print(f"Expected Data: {EXPECTED_1}")

# Assertion
assert encoded_data_1 == EXPECTED_1, "Encoded data does not match the expected output."

# Decoding Test
decoded_data_1 = decode(encoded_data_1)
print(f"Decoded Data: {decoded_data_1}")
assert decoded_data_1 == INPUT_1, "Decoded data does not match the original input."


Encoding string: integer -> b'\x03\x07integer'
Encoding integer: 12345 -> b'\x0190\x00\x00\x00\x00\x00\x00'
Encoding string: null -> b'\x03\x04null'
Encoding string: octets -> b'\x03\x06octets'
Encoding bytes: b'\x01\x02\x03' -> b'\x02\x03\x01\x02\x03'
Encoding dictionary: {'null': None, 'octets': b'\x01\x02\x03', 'integer': 12345} -> b'\x05\x03\x03\x07integer\x0190\x00\x00\x00\x00\x00\x00\x03\x04null\x00\x03\x06octets\x02\x03\x01\x02\x03'

Encoded Data: b'\x05\x03\x03\x07integer\x0190\x00\x00\x00\x00\x00\x00\x03\x04null\x00\x03\x06octets\x02\x03\x01\x02\x03'
Expected Data: b'\x05\x03\x03\x07integer\x0190\x00\x00\x00\x00\x00\x00\x03\x04null\x00\x03\x06octets\x02\x03\x01\x02\x03'
Decoded Data: {'integer': 12345, 'null': None, 'octets': b'\x01\x02\x03'}
